In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


import numpy as np

import random
import math
import time
import pandas as pd
import scipy.io
import torch


In [19]:
data = np.load('data/07 200user.npy')

In [20]:
data.shape

(23328, 5)

In [21]:
p_base = []
for i in range(243):
    p_base.append(data[i*96:(i+1)*96,0])


In [23]:
tariff = data[:96,3].tolist()


In [24]:
#生成up和down不同的96组信号
down = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(-1)
    down.append(row)
down = np.array(down)

up = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(1)
    up.append(row)
up = np.array(up)

# 生成一天的灵活度
def flexoneday(p_base, t_base, dr_sig, userpattern):
    p_flex = np.zeros(96 , dtype=int)
    for i in range(200):
        flex = np.zeros(96, dtype=int).tolist()
        temp = t_base[i]
        p = p_base[i]
        pattern = userpattern[i]
        for j in range(96 - 1):
            if dr_sig[j] == 0:
                flex[j] = 0
            elif dr_sig[j] == -1:
                flex[j] = p[j]
                p[j] = 0
                if pattern[j] == 1 or pattern[j] == 2 or pattern[j] == 5:
                    temp[j + 1] = temp[j] - 1.142
                elif pattern[j] == 3:
                    temp[j + 1] = temp[j] - 8.14
                else:
                    temp[j + 1] = temp[j] - 15.143
                if temp[j] <= 60:
                    flex[j] = 0  # 温控信号与dr信号对冲，此后灵活度均降为零
            else:  # 正1信号代表此刻功率变为最大功率
                flex[j] = 3 - p[j]
                p[j] = 3
                if pattern[j] == 1 or pattern[j] == 2 or pattern[j] == 5:
                    temp[j + 1] = temp[j] + 6.4
                elif pattern[j] == 3:
                    temp[j + 1] = temp[j] - 4.6
                else:
                    temp[j + 1] = temp[j] - 8.6
                if temp[j] >= 100:
                    flex[j] = 0  # 温控信号与dr信号对冲，灵活度降为零
        p_flex = p_flex + np.array(flex)
    return p_flex


In [25]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [26]:
ZP1 = np.load('data/ZP1.npy')
ZP = np.load('data/ZP.npy')
ZT = np.load('data/ZT.npy')
pattern200 = np.load('data/0519 2000userpattern.npy')[:200, 1:]
print(ZP1.shape,ZT.shape,pattern200.shape)

(200, 23328) (200, 23328) (200, 23328)


In [27]:
# real_flex_down = []
# for i in range(29):
#     flex1 = []
#     for j in range(96):
#         p = ZP1[:, 243*96-96*(29-i) : 243*96-96*(28-i)].copy()# 这一步很关键，否则函数会直接对ZP1进行修改，无法循环
#         t = ZT[:, 243*96-96*(29-i) : 243*96-96*(28-i)].copy()# 同上
#         a = down[j]
#         flex = flexoneday(p, t , a, pattern200[:,243*96-96*(29-i) : 243*96-96*(28-i)]).tolist()
#         flex1.append(flex)
#     real_flex_down.append(flex1)
# real_flex_down = np.array(real_flex_down)

In [28]:
ext_pbase = []
ext_ds = []
ext_pflex = []
ext_tariff = []

p = ZP1[:, 96: 2* 96].copy()
t = ZT[:, 96: 2* 96].copy()
pattern = pattern200[:, 96: 2* 96]
a = up[3].tolist()
flex = flexoneday(p, t, a, pattern).tolist()
ext_pflex += flex
ext_pbase += data[96: 2* 96,0].tolist()
ext_tariff += tariff
ext_ds += a

p = ZP1[:, 96: 2* 96].copy()
t = ZT[:, 96: 2* 96].copy()
a = down[3].tolist()
pattern = pattern200[:, 96: 2* 96]
flex = flexoneday(p, t, a, pattern).tolist()
ext_pflex += flex
ext_pbase += data[96: 2* 96,0].tolist()
ext_tariff += tariff


In [29]:
ds_0 = np.zeros(96, dtype=int).tolist()
ext_pbase = []
ext_ds = []
ext_pflex = []
ext_tariff = []
for i in range(242):
    for j in range(96):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        pattern = pattern200[:, i*96:(i + 1)*96]
        a = up[j].tolist()
        flex = flexoneday(p, t, a, pattern).tolist()
        ext_pflex += flex
        ext_pbase += data[i*96:(i + 1)*96,0].tolist()
        ext_tariff += tariff
        ext_ds += a
    for j in range(96):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        a = down[j].tolist()
        pattern = pattern200[:, i*96:(i + 1)*96]
        flex = flexoneday(p, t, a, pattern).tolist()
        ext_pflex += flex
        ext_pbase += data[i*96:(i + 1)*96,0].tolist()
        ext_tariff += tariff
        ext_ds += a
    for j in range(14):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        pattern = pattern200[:, i*96:(i + 1)*96]
        flex = flexoneday(p, t, ds_0, pattern).tolist()
        ext_pflex += flex
        ext_pbase += data[i*96:(i + 1)*96,0].tolist()
        ext_tariff += tariff
        ext_ds += ds_0
ext_pbase = np.array(ext_pbase)
ext_ds = np.array(ext_ds)
ext_pflex = np.array(ext_pflex)
ext_tariff = np.array(ext_tariff)

In [114]:
96*96*2+14*96

19776

In [30]:
print(ext_pbase.shape,ext_ds.shape,ext_pflex.shape,ext_tariff.shape)

(4785792,) (4785792,) (4785792,) (4785792,)


In [31]:
4785792/96

49852.0

In [32]:
data1={'p_norm':ext_pbase,'p_flex':ext_pflex,'ls':ext_ds,'tariff':ext_tariff}
df = pd.DataFrame(data1)
df.to_csv('data/0803 extended data 3.csv')

In [43]:
data = np.array(pd.read_csv('data/0803 extended data 3.csv')[["ls", "tariff", "p_flex"]])[206*96*2:,]
power = np.array(pd.read_csv('data/0728 200user.csv')[["p_norm", "tariff"]])[:-96*3, ] 

In [41]:
print(data.shape,power.shape)

(4746240, 3) (23040, 2)


In [42]:
23040/96

240.0

In [36]:
4746240/96/206


240.0

In [75]:
data2 = np.array(df)
np.save('data/0803 extended data 2',data2)

In [52]:
c = np.load('data/0802 extended data.npy')

In [63]:
a = [1,2,3,4,5,6,7,8,9,10]
a[-2:]

[9, 10]

In [70]:
a=np.array((2,3,4),(3,4,5))
a

TypeError: data type not understood